In [1]:
#------------------------------------------------------------------------------------------------------------------------------
# import dependencies
#------------------------------------------------------------------------------------------------------------------------------
import re
import json
import sys, os

import requests
import warnings
import psycopg2

from pprint import pprint
from copy import deepcopy

from country_list import countries_for_language

In [2]:
#------------------------------------------------------------------------------------------------------------------------------
# generate artist list
#------------------------------------------------------------------------------------------------------------------------------
"""get artists list csv"""
def get_artists(countries):  
    dicy = {}
    limit = 1000    # SPECIFY
    urlraw = r'https://ws.audioscrobbler.com/2.0/?method=geo.gettopartists&country={}&api_key=70894b101e887cf7fd9443bd5067f675&format=json&limit={}'
    
    for cty in countries.values():
        r = requests.get(urlraw.format(cty, limit))
        tmp = r.json()
#         pprint(tmp)

        try:
            for i in range(len(tmp['topartists']['artist'])):
                artistname = tmp['topartists']['artist'][i]['name']

                dicy[artistname] = 1
        except:
            print(f'top artists for {cty} CANNOT be found')
            pass
        
    return dicy


def create_glbartist():
    countriesDict = dict(countries_for_language('en'))
#     print(countriesDict)
    dicy = get_artists(countriesDict)
    # print(len(dicy), dicy)

    with open('glbartists.csv', 'w', encoding='utf-8') as fw:
        print('artists', file=fw)

        for artistname in dicy:
            print(artistname, file=fw)
            print(f'created... {artistname}')

In [3]:
create_glbartist()

top artists for Antigua & Barbuda CANNOT be found
top artists for Bosnia & Herzegovina CANNOT be found
top artists for British Virgin Islands CANNOT be found
top artists for Brunei CANNOT be found
top artists for Caribbean Netherlands CANNOT be found
top artists for Congo - Brazzaville CANNOT be found
top artists for Congo - Kinshasa CANNOT be found
top artists for Côte d’Ivoire CANNOT be found
top artists for Czechia CANNOT be found
top artists for Eswatini CANNOT be found
top artists for Falkland Islands CANNOT be found
top artists for Heard & McDonald Islands CANNOT be found
top artists for Hong Kong SAR China CANNOT be found
top artists for Iran CANNOT be found
top artists for Laos CANNOT be found
top artists for Libya CANNOT be found
top artists for Macao SAR China CANNOT be found
top artists for Micronesia CANNOT be found
top artists for Myanmar (Burma) CANNOT be found
top artists for North Korea CANNOT be found
top artists for North Macedonia CANNOT be found
top artists for Pale

KeyboardInterrupt: 

In [5]:
#------------------------------------------------------------------------------------------------------------------------------
# populate artist details
#------------------------------------------------------------------------------------------------------------------------------
"""get artist info"""
def get_artist_info(artist):
    dicy = {}
    pot = {}

    urlraw = r'https://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={}&api_key=70894b101e887cf7fd9443bd5067f675&format=json'

    try:
        r = requests.get(urlraw.format(artist))
        tmp = r.json()
#         pprint(tmp)

        artistgenre = tmp['artist']['tags']['tag']
#         print(artist, artistgenre)
    except:
        print(f"""your artist ({artist}) is WRONG""")
        pass
    else:
        for i in range(len(artistgenre)):
            if artistgenre[i]['name'] != artist:
                addartist = artistgenre[i]['name'].replace('-', ' ').title()

                pot[addartist] = 1

        genre = ', '.join(pot)

        playcount = tmp['artist']['stats']['playcount']

        listeners = tmp['artist']['stats']['listeners']

        try:
            playcountperlistener = str(int(int(playcount)/int(listeners)))
        except:
            playcountperlistener = ''

        artistinfo = re.sub(rf'(?= <a).*?(?=$)', '', tmp['artist']['bio']['summary']).replace('\n', ' ').strip()
        artistinfo = artistinfo.replace('|', ',').replace('"', '').replace("'", "")
        
        artistlink = tmp['artist']['bio']['links']['link']['href']

    #     print(genre)
    #     print(playcount)
    #     print(listeners)
    #     print(artistinfo)
    #     print(artistlink)

        return f'{genre}|{playcount}|{listeners}|{playcountperlistener}|{artistinfo}|{artistlink}'


def create_glb_artistbio():
    fw = open('glbartistsbio.csv', 'w', encoding='utf-8')

    with open('glbartists.csv', 'r', encoding='utf-8') as fr:
        print('artistname|genre|playcount|listeners|playcountperlistener|artistinfo|artistlink', file=fw)

        next(fr)

        for artistname in fr:
            artistname = artistname.strip()
    #         print(artistname)

            info = get_artist_info(artistname)
        
            artistname = artistname.replace('|', ',').replace('"', '').replace("'", "")
        
            if info != None:
                artistbio = f'{artistname}|{info}'
                print(artistbio, file=fw)

                print(f'completed... {artistname}')

    fw.close()

In [7]:
create_glb_artistbio()

completed... The xx
completed... Kanye West
completed... The Beatles
completed... Radiohead
completed... Lana Del Rey
completed... Pink Floyd
completed... Drake
completed... Metallica
completed... Nirvana
completed... Kendrick Lamar
completed... Katy Perry
completed... Led Zeppelin
completed... blink-182
completed... Depeche Mode
completed... The Rolling Stones
completed... Oasis
completed... Michael Jackson
completed... Deftones
completed... Fleetwood Mac
completed... Sia
completed... Massive Attack
completed... Calvin Harris
completed... Johnny Cash
completed... Disturbed
completed... Boards of Canada
completed... Beck
completed... alt-J
completed... Incubus
completed... M83
completed... Bruce Springsteen
completed... Hans Zimmer
completed... Moby
completed... Bonobo
completed... Shakira
completed... Foals
completed... Frank Sinatra
completed... Justin Timberlake
completed... Elvis Presley
completed... Shawn Mendes
completed... Jimi Hendrix
completed... Air
your artist (Simon & Garfu

KeyboardInterrupt: 

In [6]:
# get_artist_info('#1 Dads')

In [9]:
#------------------------------------------------------------------------------------------------------------------------------
# high lv fns
#------------------------------------------------------------------------------------------------------------------------------
def search_sim_artist(artist):
    params = {
        'host': 'localhost',
        'database': 'artistsbio_db',
        'user': 'postgres',
        'password': 'Cw18745.,'
    }
    conn = psycopg2.connect(**params)

    with conn:
#         sql = f"select artistname from public.glbartistbio where artistname ~* '{artist}' limit 5;"
        sql = f"select artistname from glbartistbio where artistname ~* '{artist}' and artistname !~* 'feat' \
            order by playcount::int desc limit 5;"
        
        cur = conn.cursor()
        cur.execute(sql)

    print('ARE YOU SEARCHING FOR?')
    for r in cur:
        artistname = r[0]
        print(r[0])

    cur.close()

    conn.commit()
    conn.close()
    
    
def search_artist_bio(artist):
    params = {
    'host': 'localhost',
    'database': 'artistsbio_db',
    'user': 'postgres',
    'password': 'Cw18745.,'
    }
    conn = psycopg2.connect(**params)

    with conn:
#         sql = f"select artistname from public.glbartistbio where artistname ~* '{artist}' limit 5;"
        sql = f"select * from glbartistbio where artistname ~* '{artist}' and artistname !~* 'feat' \
            order by playcount::int desc limit 5;"
        
        cur = conn.cursor()
        cur.execute(sql)

        rec = [r for r in cur]
        
        cur.close()
        
        for i in range(len(rec)):
            print('-' * 100)
            print(f"""\
INTRO TO YOUR ARTIST - {rec[i][1]}:
Genres: {rec[i][2]}
Playcounts: {int(rec[i][3]):,}
Listeners: {int(rec[i][4]):,}
Playcounts per listener: {rec[i][5]}
Bio: {rec[i][6]}
{rec[i][7]}""")
            
    print('-' * 100)
    
    conn.commit()
    conn.close()

In [10]:
search_sim_artist('psy')

ARE YOU SEARCHING FOR?
Gipsy Kings
The Psychedelic Furs
Psy
Psyclon Nine
Motorpsycho


In [12]:
search_artist_bio('psy')

----------------------------------------------------------------------------------------------------
INTRO TO YOUR ARTIST - Gipsy Kings:
Genres: Flamenco, Latin, Spanish, Guitar, World
Playcounts: 9,202,471
Listeners: 665,418
Playcounts per listener: 13
Bio: Gipsy Kings is a music group from Arles and Montpellier, France. They explained the evolution of their sound in the 1996 PBS documentary of their lives and music Tierra Gitana (Gipsy Land). Young brothers Nicolas, Canut and Paul Reyes accompanied their father, famed flamenco singer Jose Reyes, who started out singing cante jondo (deep chant), traditional flamenco with long-running themes of passion, love, death, etc. But they began playing rumba flamenca because we liked to watch pretty girls dance, said Nicolas.
https://last.fm/music/Gipsy+Kings/+wiki
----------------------------------------------------------------------------------------------------
INTRO TO YOUR ARTIST - The Psychedelic Furs:
Genres: New Wave, Post Punk, 80S, Al